In [1]:
# Some helper functions.

import numpy as np
from numpy.polynomial import Polynomial
from numpy.polynomial.polynomial import polypow
from math import floor

# Mean and standard deviation of a Gaussian distribution that will be used in the followings.
# Sigma is a "suggested" value for safe BFV, see https://eprint.iacr.org/2019/939.pdf
mu, sigma = 0, 3.2

def Poly(coeffs):
    """
    Helper function to build polynomials, passing a dictionary of coefficients.
    For example, passing {0: 1, 1: -2, 2: 2} returns the polynomial
    2X**2 - 2X + 1
    """
    max_power = max(coeffs.keys())
    _coeffs = np.zeros(max_power + 1)
     
    for i, c in coeffs.items():
        _coeffs[i] = c
        
    return Polynomial(_coeffs)

def pr(p):
    """ 
    Helper function to pretty-print the polynomials, with human order
    of powers, removed trailing .0, etc.
    """
    coefs = p.coef
    res = ""
    
    powers = range(len(coefs)-1, -1, -1)
    for power, coeff in zip(powers, reversed(coefs)):
        if coeff == 0:
            continue
        
        if int(coeff) == coeff:
            coeff = int(coeff)
                  
        sign = "- " if coeff < 0 else "+ "
        
        if power == 0:
            value = abs(coeff)
        elif abs(coeff) != 1:
            value = abs(coeff)
        else:
            value = ""

        power_sign = {0: "", 1: "X"}
        def_power_sign = f"X**{power}"
        
        res += f" {sign}{value}{power_sign.get(power, def_power_sign)}"
        
    if res[1] == "+":
        return res[3:]
    if res[1] == "-":
        return res[1:]

def mod_on_coefficients(polynomial, modulo):
    """
    Apply the modulo on the coefficients of a polynomial.
    """
    coefs = polynomial.coef
    mod_coefs = []
    for c in coefs:
        mod_coefs.append(c % modulo)
        
    return Polynomial(mod_coefs)

def round_on_nearest_integer(polynomial):
    """
    Round the coefficients of a polynomial to the
    nearest integer.
    """
    coefs = polynomial.coef
    round_coefs = []
    for c in coefs:
        round_coefs.append(round(c))
    
    return Polynomial(round_coefs)

Main sources:
  - A Homomorphic Encryption Illustrated Primer - https://blog.n1analytics.com/
  - SEAL Manual (https://www.microsoft.com/en-us/research/uploads/prod/2017/11/sealmanual-2-3-1.pdf)

# BFV Homomorphic encryption scheme
The Brakerski-Fan-Vercauteren (BFV) scheme is an homomorphic encryption (HE) scheme which allows the computation of some functions directly on *encrypted data*. 

In particular, it allows the computation of additions and multiplication:

- between ciphertexts and ciphertexts;
- between ciphertexts and plaintexts.


The result of the operations, once decrypted, is the same as if it were applied on the corresponding plaintexts.

![he](images/HE.png)

The schemes like BFV or also CKKS are based on a hard computation problem called *Ring Learning With Errors*.

Let's understand this problem starting from the bottom.

## Data representation
The data in these schemes is represented by *polynomials*. This holds both when data is encrypted (ciphertexts) and when it is unencrypted (plaintexts).

So, the starting point are common polynomials like:

In [2]:
p = Poly({0: 4, 1: 2, 2: 1})
print(pr(p))

X**2 + 2X + 4


However, there is more. 

### Coefficients
First of all, the coefficients of the polynomials are whole numbers, and are the *remainder* relative to some other number $k$ (that is $\bmod k$).

Take for example $k=12$; we can see the ring as a common clock, in which each our corresponds to an element in the ring.
Adding $3$ to $11$ will result in $2$ instead of $14$.

![modularNumbers](images/bfv/ModularNumbers.png)

It also important to note that in this ring we could make the numbers go from $-5$ to $6$: this is arbitrary, because it is easy to see that a remainder of $-1$ is the same as a remainder of $11$. In the following this consideration will be assumed.

### Polynomial modulus
The second aspect is that also the polynomials themselves are remainder: we define a special polynomial called *polynomial modulus*. Each polynomial used in the scheme is divided by this polynomial modulus and the remainder is taken.

In the HE schemes it is common to take this polynomial modulus in the form $x^n + 1$ where $n$ is a power of $2$. For example, if we take $n=8$ we have the polynomial $x^{8} + 1$.

Put simply, the polynomials employed in the scheme have the following two characteristics:

  1. Their coefficients are modulo $k$, creating a ring from $0$ to $k$;
  2. The maximum degree is $n-1$;

So, all the polynomials we are considering are of the form (assuming $n=8$):
$$ a_7 x^7+a_6 x^6+a_5 x^5+a_4 x^4+a_3 x^3+a_2 x^2+a_1 x+a_0 $$

remembering that each of the 8 coefficients $a_i$ can range from $0$ to $k-1$.

We can also give a powerful graphical representation of this:

![immagine.png](images/bfv/PolynomialTorus.png)

Each loop contains the 12 possible values of the coefficient of one of the powers of $x$ that appears in the polynomial. 

Formally: the polynomials used in the scheme are in the space $R_k = \mathbb{Z}_k[x] / (x^n + 1)$.

### Practical examples
Let's see this in action. In HE, there are a lot of multiplications between polynomials.
Let $n = 8$ (hence, the polynomial modulus is $x^8 + 1$), and $k = 7$.
When we multiply two powers of $x$, for example $3 x^{4}$ and $4x^5$ we add their exponent, making $12 x^{9}$. 

After each operation, we have to perform the division with the polynomial modulus and keep the remainder.

$$ 12 x^9 \bmod x^8 + 1 = -12x $$

Also, the result's coefficients have to be taken $\bmod k$.

$$ [-12x]_k = 2x $$

In [3]:
a = Poly({4: 3})
b = Poly({5: 4})
polynomial_modulus = Poly({0: 1, 8: 1})

prod = a * b

quo, rem = divmod(prod, polynomial_modulus)
final_result = mod_on_coefficients(rem, 7)

print(f"A: {pr(a)}")
print(f"B: {pr(b)}")
print(f"Product of A and B: {pr(prod)}")
print(f"Polynomial modulus: {pr(polynomial_modulus)}")
print(f"----------------------")
print(f"Remainder of (A*B) / polynomial modulus: {pr(rem)}")
print(f"Apply also mod k: {pr(final_result)}")

A: 3X**4
B: 4X**5
Product of A and B: 12X**9
Polynomial modulus: X**8 + 1
----------------------
Remainder of (A*B) / polynomial modulus: - 12X
Apply also mod k: 2X


The polynomials used as modulo, $x^n + 1$ are in the family of *cyclotomic polynomials*.

## Encryption using polynomials ring
Now we can go to the encryption/decryption phase. And we start this discussing how private and public keys are created.

### Private and public keys
Encryption takes a *plaintext* and converts it to a *ciphertext* using a public key which is derived from a private key.
The decryption - the opposite process - is feasible only if you know the private key.

Let's write if formally for the case of BFV.

A plaintext is a polynomial from the ring with polynomial modulus $x^n + 1$, with $n$ being a power of $2$, and coefficients modulus $p$ (which will be denoted as *plaintext coefficient modulus*). The polynomial modulus is denoted with $\Phi_n$.

The encryption of a plaintext is a ciphertext which is represented by (at least) **two** polynomial from the ring with the same polynomial modulus, but with the coefficients modulus $q$ (which will be denoted as *ciphertext coefficient modulus*) which is typically much larger than $p$.

In real cases of BFV applications, it is common to take values of $n$ of at least $2048$ while for $q$, usually very large values are used.

To make a simpler example, we will use much smaller values, for example $n=16$, $p=7$ and $q=874$. Note that these parameter are not *secure* for real cases (and we will see why...).

In [4]:
n = 16
polynomial_modulus = Poly({0: 1, n: 1})
p = 7
q = 874

print(f"Plaintext coefficient modulo: {p}")
print(f"Ciphertext coefficient modulo: {q}")
print(f"Polynomial modulus: {pr(polynomial_modulus)}")

Plaintext coefficient modulo: 7
Ciphertext coefficient modulo: 874
Polynomial modulus: X**16 + 1


Let's generate the private key, $k_s$: the secret key corresponds to a random polynomial with coefficients of either $-1, 0, 1$. 

For example:

In [1]:
coeffs = {}
for i in range(0, 16):
    coeffs[i] = np.random.randint(-1, 2)
    
s = Poly(coeffs)
print(f"Secret key: {pr(s)}")

NameError: name 'np' is not defined

Next we generate a public key.

For this we need:

  1. a random polynomial from the ciphertext space, hence with coefficients modulo $q$, which we'll call $a$;
  2. an *error polynomial*, which is small in the sense that has small coefficients drawn from a discrete Gaussian distribution. We will use this once, then it is discarded. We'll call this $e$.

At this point we can define the public key as the pair of polynomials:

$$k_p = ([{-(a k_s + e)}]_{\Phi_n, q} , a)$$

where the polynomial arithmetic is done modulo the polynomial modulus and the coefficient arithmetic modulo $q$.

In [6]:
coeffs = {}
for i in range(0, n):
    coeffs[i] = np.random.randint(0, q)

a = Poly(coeffs)

# this should be bounded
dist = np.random.default_rng().normal(mu, sigma, 16)
coeffs = {}
for ind in range(0, n):
    coeffs[ind] = round(dist[ind])


e = Poly(coeffs)

pk_0 = -((a * s) + e)
pk_0 = divmod(pk_0, polynomial_modulus)[1]
pk_0 = mod_on_coefficients(pk_0, q)

pk_1 = a

pk = (pk_0, pk_1)

print(f"a: {pr(a)}\n")
print(f"e: {pr(e)}\n")
print(f"Public key: {pr(pk_0)}, {pr(pk_1)}\n")

a: 741X**15 + 200X**14 + 56X**13 + 720X**12 + 526X**11 + 233X**10 + 761X**9 + 271X**8 + 495X**7 + 291X**6 + 77X**5 + 139X**4 + 200X**3 + 168X**2 + 182X + 670

e: - X**15 - 2X**14 - X**13 + 4X**12 - X**11 + X**10 + 4X**9 - 3X**8 - 3X**7 - 4X**6 + 2X**5 - X**4 + 2X**3 - 8X**2 - 2X - 1

Public key: 755X**15 + 665X**14 + 234X**13 + 542X**12 + 659X**11 + 838X**10 + 320X**9 + 654X**8 + 152X**7 + 543X**6 + 663X**5 + 449X**4 + 833X**3 + 681X**2 + 248X + 596, 741X**15 + 200X**14 + 56X**13 + 720X**12 + 526X**11 + 233X**10 + 761X**9 + 271X**8 + 495X**7 + 291X**6 + 77X**5 + 139X**4 + 200X**3 + 168X**2 + 182X + 670



It is interesting to note that, in a sense, the secret key is contained in the public key. However, to mask it, it is multiplied with the polynomial $a$ which has coefficients in the range $\bmod q$ (it is quite "big"). 

However, also $a$ is contained in the public key; a little error $e$ is added to $a*s$ and this makes it computationally hard to retrieve $s$. 
This is the exact definition of the *Ring Learning with Errors*.

In our toy example it would be possible to retrieve $s$ using a brute force attack. trying every possible $s$ and checking for similarity with the first term of the public key. There are only $3^{16} = 43046721$ combinations; it's obvious that with bigger $n$ the combinations can become, for example $3^{4096}$ which is *quite* large. However, the security is tested also against other type of more clever attacks.

To recap:

$$ s \rightarrow \text{"small" polynomial (coefficients in -1, 0, 1)}$$
$$ a \rightarrow \text{"big polynomial (coefficients in mod q)}$$
$$ e \rightarrow \text{"small noise" polynomial (coefficients drawn by a Gaussian distribution)}$$

$$ k_s$$
$$ k_p = \left ( \left [ -(a k_s + e) \right ]_{\Phi_n, q} , a \right ) $$

### Encryption
Encryption transforms a plaintext - encoded in the form of a polynomial with coefficients modulo $p$ (that, is in the space $R_p = \mathbb{Z}_p[{x}]/(x^n + 1)$ into a ciphertext.
A fresh ciphertext, in BFV, is a pair of polynomials with coefficients modulo $q$ (that is, they are in the space $R_{\Phi_n, q} = \mathbb{Z}_{\Phi_n, q}[{x}]/(x^n + 1)$). 

In [7]:
m = Poly({0: 1, 1: 1, 2: 1})
print(f"Plain message: {pr(m)}")

Plain message: X**2 + X + 1


To perform the encryption we need three "small" polynomials:

1. Two error polynomials ("small error" polynomials), extracted from a discrete Gaussian distribution (similarly to the one used in the public key);
2. A "small" polynomial, $u$, which has coefficients drawn from (-1, 0, 1), similar to $s$.

In [8]:
dist = np.random.default_rng().normal(mu, sigma, 16)
coeffs = {}
for ind in range(0, 16):
    coeffs[ind] = round(dist[ind])

e_1 = Poly(coeffs)

dist = np.random.default_rng().normal(mu, sigma, 16)
coeffs = {}
for ind in range(0, 16):
    coeffs[ind] = round(dist[ind])

e_2 = Poly(coeffs)

coeffs = {}
for i in range(0, 16):
    coeffs[i] = np.random.randint(-1, 2)
    
u = Poly(coeffs)

print(f"e_1: {pr(e_1)}\n")
print(f"e_2: {pr(e_2)}\n")
print(f"u: {pr(u)}\n")

e_1: 2X**15 + 5X**14 + 4X**12 + 6X**11 - X**10 - 2X**9 - 3X**8 - X**7 - 3X**6 + 3X**5 + 5X**3 + X**2 + 5X + 2

e_2: 2X**15 - 7X**14 - 2X**13 - X**12 + 3X**11 + X**10 - 3X**9 + 3X**8 + 6X**7 + X**6 - X**5 - X**4 - 5X**3 + 2X - 1

u: - X**13 - X**12 - X**11 - X**6 - X**3 - X**2



We use these polynomials in the encryption phase, then we discard them.
The ciphertext is represented by two polynomials calculated as:
$$ct = \left ( \left [ k_{p_0} u + e_1 + \left \lfloor \frac{q}{p} \right \rfloor m \right ]_{\Phi_n, q} , \left [k_{p_1} u + e_2 \right ]_{\Phi_n, q} \right)$$

Which can be rewritten, expanding $k_p$:
$$ct = \left ( \left [ -asu -eu + e_1 + \left \lfloor \frac{q}{p} \right \rfloor m \right ]_{\Phi_n, q} , \left [ au + e_2 \right ]_{\Phi_n, q} \right)$$

The initial message had coefficients in the range of $\bmod p$. However, they have been *scaled* by $\left \lfloor \frac{q}{p} \right \rfloor$. This means that, in the ciphertext, the message has coefficients in the range of $\bmod q$. 
Morover the role of $u$ become clear: $u$ is a mask which changes at every encryption, and is responsible for the property of confusion typical of HE schemes. The same plaintext message will always have a different encryption.

The other terms in the ciphertext *hide* the scaled message, including $-asu$ (which is the biggest one, in terms of coefficients).
On the other hand, having $-asu$ and $au$ in the ciphertext is the key for the decryption phase.

In [9]:
ct0 = pk[0] * u + e_1 + floor((q/p)) * m
ct0 = divmod(ct0, polynomial_modulus)[1]
ct0 = mod_on_coefficients(ct0, q)

ct1 = pk[1] * u + e_2
ct1 = divmod(ct1, polynomial_modulus)[1]
ct1 = mod_on_coefficients(ct1, q)

print(f"Ciphertext: {pr(ct0)}, {pr(ct1)}")

Ciphertext: 439X**15 + 758X**14 + 322X**13 + 836X**12 + 67X**11 + 398X**10 + 785X**9 + 419X**8 + 678X**7 + 810X**6 + 182X**5 + 862X**4 + 744X**3 + 436X**2 + 367X + 312, 580X**15 + 548X**14 + 346X**13 + 351X**12 + 39X**11 + 93X**10 + 861X**9 + 769X**8 + 213X**7 + 512X**6 + 763X**5 + 502X**4 + 256X**3 + 780X**2 + 228X + 30


### Decryption
The decryption is relatively simple.

In fact, you may have noted that the biggest error term, which hides our message in the ciphertext, is given by $-asu$.
However, if we are the entity who crypted the message:

  1. we have $s$;
  2. we have $au$, because it is contained in the second term of the ciphertext.

So, we just have to multiply the second term of the ciphertext with $s$, and sum it with the first term of the ciphertext:

$$ [{ct_0 + ct_1s}]_{\Phi_n, q} = \left [ -asu -eu + e_1 + \left \lfloor \frac{q}{p} \right \rfloor m + asu + se_2 \right ]_{\Phi_n, q} $$

We end up with:

$$ \left [ \left \lfloor \frac{q}{p} \right \rfloor m -eu + e_1 + se_2 \right]_{\Phi_n, q} $$

Inside this polynomial we have the scaled message summed to some noise. If the noise is not too big, we can recover the message.

To do that, we just try to make the modulo with the polynomial modulus, than to apply $\bmod q$ to the coefficients of the resulting polynomial.

In [10]:
decrypt = ct1 * s + ct0
decrypt = divmod(decrypt, polynomial_modulus)[1]
decrypt = mod_on_coefficients(decrypt, q)

print(f"Scaled plaintext + errors: {pr(decrypt)}")

Scaled plaintext + errors: 869X**15 + 17X**14 + 13X**13 + 3X**12 + 7X**11 + 873X**10 + 865X**9 + 854X**8 + 841X**7 + 842X**6 + 16X**5 + 18X**4 + 14X**3 + 130X**2 + 141X + 137


If we rescale...

In [11]:
decrypt = decrypt * p/q
print(f"De-scaled plaintext plus errors: {pr(decrypt)}")

De-scaled plaintext plus errors: 6.959954233409611X**15 + 0.13615560640732266X**14 + 0.10411899313501144X**13 + 0.02402745995423341X**12 + 0.05606407322654462X**11 + 6.991990846681922X**10 + 6.9279176201373X**9 + 6.839816933638444X**8 + 6.7356979405034325X**7 + 6.74370709382151X**6 + 0.12814645308924486X**5 + 0.14416475972540047X**4 + 0.11212814645308924X**3 + 1.0411899313501145X**2 + 1.1292906178489703X + 1.097254004576659


Don't forget to round to the nearest integer and make the modulo $\bmod p$!

In [12]:
decrypt = round_on_nearest_integer(decrypt)
decrypt = mod_on_coefficients(decrypt, p)

print(f"Plain starting message: {pr(m)}")
print(f"Final decryption result: {pr(decrypt)}")

Plain starting message: X**2 + X + 1
Final decryption result: X**2 + X + 1


This is exactly our message!

If the coefficients are too noisy, they will be rounded to another integer, and we will end up with wrong terms. The amount of room for noise can be adjusted by making the ratio $q/p$ larger or smaller.

We are ready to define some other helper functions to automatize the process of encryption and decryption.

In [13]:
def generate_keys(n, q):
    """
    Generate public and secret key, using the parameters
    d, q passed as parameters.
    
    Returns sk, pk.
    """
    polynomial_modulus = Poly({0: 1, n: 1})
    coeffs = {}
    
    # Secret key
    for i in range(0, n):
        coeffs[i] = np.random.randint(-1, 2)

    sk = Poly(coeffs)
    
    # Public key
    coeffs = {}
    for i in range(0, n):
        coeffs[i] = np.random.randint(0, q)

    a = Poly(coeffs)

    dist = np.random.default_rng().normal(mu, sigma, 16)
    coeffs = {}
    for ind in range(0, n):
        coeffs[ind] = round(dist[ind])

    e = Poly(coeffs)

    pk_0 = -((a * sk) + e)
    pk_0 = divmod(pk_0, polynomial_modulus)[1]
    pk_0 = mod_on_coefficients(pk_0, q)

    pk_1 = a

    pk = (pk_0, pk_1)
    
    return sk, pk

def encrypt_poly(m, pk, n, p, q):
    """
    Encrypt a polynomial message given the public key.
    Returns the encrypted polynomial.
    """
    polynomial_modulus = Poly({0: 1, n: 1})

    dist = np.random.default_rng().normal(mu, sigma, 16)
    coeffs = {}
    for ind in range(0, n):
        coeffs[ind] = round(dist[ind])

    e_1 = Poly(coeffs)

    dist = np.random.default_rng().normal(mu, sigma, 16)
    coeffs = {}
    for ind in range(0, n):
        coeffs[ind] = round(dist[ind])

    e_2 = Poly(coeffs)

    coeffs = {}
    for i in range(0, n):
        coeffs[i] = np.random.randint(-1, 2)

    u = Poly(coeffs)

    ct0 = kp[0] * u + e_1 + floor((q/p)) * m
    ct0 = divmod(ct0, polynomial_modulus)[1]
    ct0 = mod_on_coefficients(ct0, q)
    
    ct1 = kp[1] * u + e_2 
    ct1 = divmod(ct1, polynomial_modulus)[1]
    ct1 = mod_on_coefficients(ct1, q)
    
    return (ct0, ct1)

def decrypt_poly(ct, ks, n, p, q):
    """
    Decrypts an encrypted polynomial, given the secret key.
    Returns the decrypted polynomial.
    """
    polynomial_modulus = Poly({0: 1, n: 1})
    decrypt = Poly({0: 0})
    
    for i, term in enumerate(ct):
        decrypt += ct[i] * polypow(ks.coef, i, 1000)
    
    decrypt = divmod(decrypt, polynomial_modulus)[1]
    decrypt = mod_on_coefficients(decrypt, q)
    
    decrypt = decrypt * p/q
    decrypt = round_on_nearest_integer(decrypt)
    decrypt = mod_on_coefficients(decrypt, p)
    
    return decrypt    

## Homomorphic operations
Obviously, we are interested in the special aspect of these HE schemes: the possibility to compute homomorphic additions and  multiplications.

This means that we can add and multiply numbers while they are still encrypted, without having to decrypt them.

### Homomorphic addition (between ciphertexts)
The simplest case is the addition of two ciphertexts. 
Starting from $ct_1$ and $ct_2$, which are the encryption of messages $m_1$ and $m_2$ respectively, we can compute $ct_3 = ct_1 + ct_2$.
The decryption of $ct_3$ is equal to $m_1 + m_2$.

Obviously, this assuming the correctness of the processes and suitable encryption parameters.

In [14]:
n = 16
p = 7
q = 874

ks, kp = generate_keys(n, q)

m1 = Poly({0: 1, 1: 1, 2: 1})
m2 = Poly({1: 1})
print(f"First message: {pr(m1)}")
print(f"Second message: {pr(m2)}")
print(f"Expected sum: {pr(m1+m2)}")
print(f"-------------------------")

enc_m1 = encrypt_poly(m1, kp, n, p, q)
enc_m2 = encrypt_poly(m2, kp, n, p, q)
print(f"First ciphertext: {pr(enc_m1[0])}, {pr(enc_m1[1])}\n")
print(f"Second ciphertext: {pr(enc_m2[0])}, {pr(enc_m2[1])}\n")

First message: X**2 + X + 1
Second message: X
Expected sum: X**2 + 2X + 1
-------------------------
First ciphertext: 122X**15 + 472X**14 + 859X**13 + 427X**12 + 44X**11 + 871X**10 + 104X**9 + 780X**8 + 83X**7 + 681X**6 + 404X**5 + 363X**4 + 717X**3 + 529X**2 + 617X + 859, 803X**15 + 133X**14 + 747X**13 + 754X**12 + 7X**11 + 396X**10 + X**9 + 505X**8 + 34X**7 + 197X**6 + 698X**5 + 431X**4 + 465X**3 + 868X**2 + 341X + 742

Second ciphertext: 721X**15 + 507X**14 + 78X**13 + 605X**12 + 442X**11 + 640X**10 + 350X**9 + 273X**8 + 176X**7 + 684X**6 + 561X**5 + 227X**4 + 194X**3 + 680X**2 + 709X + 669, 851X**15 + 862X**14 + 482X**13 + 288X**12 + 355X**11 + 185X**10 + 556X**9 + 152X**8 + 514X**7 + 634X**6 + 562X**5 + 605X**4 + 496X**3 + 91X**2 + 704X + 841



To perform the addition between the two ciphertexts, it is sufficient to add them element-wise.
In the followings $u_1$ and $u_2$, as well as $e_1, \ldots, e_4$ denote the "small" and "small errors" polynomials created during the encryption phase.

$$ enc\_m_1 + enc\_m_2 = \left ( \left [k_{p_0}u_1 + e_1 + \left \lfloor \frac{q}{p} \right \rfloor m_1 \right ]_{\Phi_n, q} , \left [k_{p_1}u_1 + e_2 \right]_{\Phi_n, q} \right ) + \left ( \left [k_{p_0}u_2 + e_3 + \left \lfloor \frac{q}{p} \right \rfloor m_2 \right ]_{\Phi_n, q} , \left [ k_{p_1}u_1 + e_4 \right]_{\Phi_n, q} \right ) $$

$$ enc\_m_1 + enc\_m_2 = \left ( \left [k_{p_0}(u_1 + u_2) + e_1 + e_3 + \left \lfloor \frac{q}{p} \right \rfloor (m_1 + m_2) \right ]_{\Phi_n, q} , \left [ k_{p_1}(u_1 + u_2) + e_2 + e_4 \right ]_{\Phi_n, q} \right ) $$

We are interested in the term $\left \lfloor \frac{q}{p} \right \rfloor(m_1 + m_2)$, which is the scaled sum of our messages; however, new noise terms appeared in the ciphertext.

Let's try to decrypt this, multiplying the second term of the ciphertext with $s$ and summing it up. Also, expand $k_p$:

$$ [{ct_0 + ct_1s}]_{\Phi_n, q} = \left [ { -as(u_1 + u_2) - e(u_1 + u_2) + e_1 + e_3 + \left \lfloor \frac{q}{p} \right \rfloor(m_1 + m_2) + (a(u_1 + u_2) + e_2 + e_4)s } \right ]_{\Phi_n, q} $$

$$ [{ct_0 + ct_1s}]_{\Phi_n, q} = \left [ -as(u_1 + u_2) - e(u_1 + u_2) + e_1 + e_3 + \left \lfloor \frac{q}{p} \right \rfloor(m_1 + m_2) + as(u_1 + u_2) + s(e_2 + e_4) \right ]_{\Phi_n, q} $$

$$ [{ct_0 + ct_1s}]_{\Phi_n, q} = \left [ -e(u_1 + u_2) + e_1 + e_3 + \left \lfloor \frac{q}{p} \right \rfloor(m_1 + m_2) + s(e_2 + e_4) \right ]_{\Phi_n, q} $$

We have some new noise terms. Before analyzing them, let's remember that the noise polynomials become a problem if one of their coefficients is larger than $\frac{q}{2p}$, which, in our case, is around $62$. This is because, during the decryption, after the scaling, the rounding to the nearest integer will round to the wrong one! According to the type of encoding used, this can result in a small or big problem.

The new noise terms are:

  1. $-e(u_1 + u_2)$: this is the product of a "small noise" polynomial with the sum of two "small" polynomials (with coefficients $-1, 0, 1$). This can be quite big, because sometimes the sum of $-1, 0, 1$ will be close to $0$, while other times it will become $2, 3, 4...$. This can reach $62$ pretty fast.
  2. $e_1 + e_3$: this is only the sum of two "small noise" polynomials. It's less worrying than the first.
  3. $s(e_2 + e_4)$: this is bad similarly to the the first.

Can we graphically visualize how bad is this?

![eu1u2](images/bfv/addition_noise/plot.png)

It is possible to see that, after just one addition, there is a decent probability that a coefficient of the noise polynomials will be higher than $62$, leading to an error in the decryption phase.

To have more room, we have to increment the ratio $q/t$.

Let's compute an addition.

In [15]:
def add_ciphertexts(enc_m1, enc_m2, n, q):
    polynomial_modulus = Poly({0: 1, n: 1})
    enc_sum_0 = enc_m1[0] + enc_m2[0]
    enc_sum_1 = enc_m1[1] + enc_m2[1]

    enc_sum_0 = divmod(enc_sum_0, polynomial_modulus)[1]
    enc_sum_0 = mod_on_coefficients(enc_sum_0, q)

    enc_sum_1 = divmod(enc_sum_1, polynomial_modulus)[1]
    enc_sum_1 = mod_on_coefficients(enc_sum_1, q)

    enc_sum = (enc_sum_0, enc_sum_1)
    return enc_sum

enc_sum = add_ciphertexts(enc_m1, enc_m2, n, q)
print(f"Encrypted sum: {pr(enc_sum[0])}, {pr(enc_sum[1])}\n")

decrypted = decrypt_poly(enc_sum, ks, n, p, q)
print(f"Expected sum: {pr(m1+m2)}")
print(f"Decrypted result: {pr(decrypted)}")

Encrypted sum: 843X**15 + 105X**14 + 63X**13 + 158X**12 + 486X**11 + 637X**10 + 454X**9 + 179X**8 + 259X**7 + 491X**6 + 91X**5 + 590X**4 + 37X**3 + 335X**2 + 452X + 654, 780X**15 + 121X**14 + 355X**13 + 168X**12 + 362X**11 + 581X**10 + 557X**9 + 657X**8 + 548X**7 + 831X**6 + 386X**5 + 162X**4 + 87X**3 + 85X**2 + 171X + 709

Expected sum: X**2 + 2X + 1
Decrypted result: X**10 + X**2 + 2X + 1


Remember that we are in a ring, so $+6 \equiv -1$, etc.
If the result is not correct, try to encrypt the messages again and re-computing the addition.

### Homomorphic multiplication (between ciphertexts)
The multiplication between ciphertexts is more complicated than the addition and can cause an important increment of the noise in the resulting ciphertext.

Before diving into the actual multiplication, we can foresee that, at some point, we will have a term: $\left \lfloor \frac{q}{p} \right \rfloor m_1 * \left \lfloor \frac{q}{p} \right \rfloor m_2$ which is $\frac{q^2}{p^2}m_1m_2$. So, we start this phase anticipating that, during the multiplication, we will multiply the ciphertexts also with $\frac{p}{q}$ in order to come back to $\left \lfloor \frac{q}{p} \right \rfloor m_1m_2$: noise apart, this is the term we will want to have in the ciphertext, so we can decrypt it later.

Another point is: let's consider the ciphertext in the form that we used in the decryption: $ct = ([{ct_0 + ct_1s}]_{\Phi_n, q}$. Basically we are expressing the ciphertexts as *polynomials in $s$*. It will soon be clear why it is useful to do this.

Let $m_1$ and $m_2$ the plain polynomials, and $enc_1$, $enc_2$ their encryption. We have:

$$ enc_1 * enc_2 = \left [{\left \lfloor \frac{p}{q}(enc_{1_0} + enc_{1_1}s) * (enc_{2_0} + enc_{2_1}s) \right \rceil} \right]_{\Phi_n, q} = $$
$$ enc_1 * enc_2 = \left [{\left \lfloor \frac{p}{q}(enc_{1_0}enc_{2_0} + enc_{1_0}enc_{2_1}s + enc_{1_1}enc_{2_0}s + enc_{1_1}enc_{2_1}s^2) \right \rceil} \right ]_{\Phi_n, q} $$

This is interesting because, if we group using $s$ and we give some simpler names to these terms, we have:
$$ c_0 = \left [{\left \lfloor \frac{p}{q} enc_{1_0}enc_{2_0} \right \rceil} \right ]_{\Phi_n, q} $$
$$ c_1 = \left [{\left \lfloor \frac{p}{q} (enc_{1_0}enc_{2_1} + enc_{1_1}enc_{2_0}) \right \rceil} \right]_{\Phi_n, q} $$
$$ c_2 = \left [{\left \lfloor \frac{p}{q} enc_{1_1}enc_{2_1} \right \rceil} \right]_{\Phi_n, q} $$

This means that, if we express the multiplication result as this three-terms ciphertext:
$$ enc_1 * enc_2 = enc_{result} = (c_0, c_1, c_2) $$

We can also rewrite the decryption operation to work also on three-terms (or... $n$-terms!) ciphertexts:
$$ m_1 * m_2 = \left [{ \left \lfloor \frac{p}{q} [{enc_{result_0}s^0 + enc_{result_1}s^1 + enc_{result_2}s^2}]_{\Phi_n, q} \right \rceil} \right ]_{\Phi_n, p} $$

#### Noise in multiplications
Unfortunately, to understand the noise propagation during the multiplications, we have to expands some of the last equations into simpler terms: $k_p$ first, then $s$, $e$, $u$... 

Luckily, we know from the decryption phase that the mask is eliminated with $-asu + asu$; so, we can omit some terms. In particular, we can "reconsider" the public key as $k_p = (e, 0)$ and speed up some calculations.

Recalling the form of our ciphertexts:
$$ enc_1 = \left ( \left [{k_{p_0}u_1 + e_1 + \left \lfloor \frac{q}{p} \right \rfloor m_1} \right ]_{\Phi_n, q}, \left [{k_{p_1}u_1 + e_2} \right ]_{\Phi_n, q} \right ) $$
$$ enc_2 = \left ( \left [{k_{p_0}u_2 + e_3 + \left \lfloor \frac{q}{p} \right \rfloor m_2} \right ]_{\Phi_n, q}, \left [{k_{p_1}u_2 + e_4} \right ]_{\Phi_n, q} \right) $$

We can rewrite each component of the decryption, one by one for clarity:

$$c_0 = \left [ \left \lfloor \frac{p}{q}( k_{p_0}u_1 + e_1 + \left \lfloor \frac{q}{p} \right \rfloor m_1)(k_{p_0}u_2 + e_3 + \left \lfloor \frac{q}{p} \right \rfloor m_2) \right \rceil \right ]_{\Phi_n, q} $$
$$ = \left [ \left \lfloor \frac{p}{q}e^2u_1u_2 - \frac{p}{q}eu_1e_3 - eu_1m_2 - \frac{p}{q}eu_2e_1 + \frac{p}{q}e_3e_1 + m_2e_1 - m_1eu_2 + m_1e_3 + \left \lfloor \frac{q}{p} \right \rfloor m_1m_2 \right \rceil \right ]_{\Phi_n, q}$$

$$c_1 = \left [ \left \lfloor \frac{p}{q}((k_{p_0}u_1 + e_1 + \left \lfloor \frac{q}{p} \right \rfloor m_1)(k_{p_1}u_2 + e_4) + (k_{p_1}u_1 + e_2)(k_{p_0}u_2 + e_3 + \left \lfloor \frac{q}{p} \right \rfloor m_2))s \right \rceil \right ]_{\Phi_n, q}$$
$$ = \left [ \left \lfloor -\frac{p}{q}eu_1e_4s + \frac{p}{q}e_1e_4s + m_1e_4s - \frac{p}{q}eu_2e_2s + \frac{p}{q}e_3e_2s + m_2e_2s \right \rceil \right ]_{\Phi_n, q}$$

$$c_2 = \left [ \left \lfloor \frac{p}{q}(k_{p_1}u_1 + e_2)(k_{p_1}u_2 + e_4)s^2 \right \rceil \right ]_{\Phi_n, q}$$
$$ = \left [ \left \lfloor \frac{p}{q}e_2e_4s^2 \right \rceil \right ]_{\Phi_n, q}$$

By experience, we can say that the most problematic terms are the one involving the messages $*$ secret key $*$ an error are the most problematic.

If we use the encryption parameters that we used in the addition case ($t=7$, $q=874$) we have a problem if a coefficient in the result polynomial will be higher than $\left \lfloor \frac{q}{2t} \right \rfloor = 62$.

Let's inspect the distribution of the coefficients in one multiplication, for all the noise terms we obtained:

![mult_noise](images/bfv/multiplication_noise/plot.png)

It is basically sure that this will not work: in the histogram for $-e * u_1 * m_2$ we see that there is a very high probability of coefficients being in the range $[{-500, 500}]$. We have to increment the ratio $\frac{q}{p}$ to have more room with noise. Let's try with $424242$:

In [16]:
n = 16
p = 7
q = 424242

ks, kp = generate_keys(n, q)

m1 = Poly({0: 1, 1: 1, 2: 3})
m2 = Poly({0: -1, 1: -2, 2: 1})
print(f"First message: {pr(m1)}")
print(f"Second message: {pr(m2)}")
print(f"Expected product: {pr(m1*m2)}")
print(f"-------------------------")

enc_m1 = encrypt_poly(m1, kp, n, p, q)
enc_m2 = encrypt_poly(m2, kp, n, p, q)
print(f"First ciphertext: {pr(enc_m1[0])}, {pr(enc_m1[1])}\n")
print(f"Second ciphertext: {pr(enc_m2[0])}, {pr(enc_m2[1])}")

First message: 3X**2 + X + 1
Second message: X**2 - 2X - 1
Expected product: 3X**4 - 5X**3 - 4X**2 - 3X - 1
-------------------------
First ciphertext: 287379X**15 + 137677X**14 + 281413X**13 + 319501X**12 + 54508X**11 + 82914X**10 + 35062X**9 + 306727X**8 + 324478X**7 + 294896X**6 + 334368X**5 + 340684X**4 + 23564X**3 + 220734X**2 + 72967X + 68241, 105497X**15 + 279692X**14 + 398992X**13 + 43305X**12 + 190834X**11 + 199360X**10 + 265798X**9 + 242377X**8 + 272047X**7 + 399949X**6 + 381047X**5 + 51756X**4 + 170892X**3 + 318242X**2 + 36391X + 361477

Second ciphertext: 328368X**15 + 21004X**14 + 309926X**13 + 274249X**12 + 36481X**11 + 107217X**10 + 202934X**9 + 137356X**8 + 65453X**7 + 381512X**6 + 107571X**5 + 181070X**4 + 383154X**3 + 109203X**2 + 219262X + 156964, 2368X**15 + 209092X**14 + 231558X**13 + 5081X**12 + 84194X**11 + 158727X**10 + 409289X**9 + 415929X**8 + 245719X**7 + 362707X**6 + 319137X**5 + 417402X**4 + 291829X**3 + 238214X**2 + 75697X + 56662


In [17]:
def multiply_ciphertexts(ct1, ct2, n, p, q):
    polynomial_modulus = Poly({0: 1, n: 1})
    
    c0 = (p/q) * ct1[0] * ct2[0]
    c0 = divmod(c0, polynomial_modulus)[1]
    c0 = mod_on_coefficients(c0, q)
    c0 = round_on_nearest_integer(c0)
    
    c1 = (p/q) * (ct1[0] * ct2[1] + ct1[1] * ct2[0])
    c1 = divmod(c1, polynomial_modulus)[1]
    c1 = mod_on_coefficients(c1, q)
    c1 = round_on_nearest_integer(c1)
    
    c2 = (p/q) * ct1[1] * ct2[1]
    c2 = divmod(c2, polynomial_modulus)[1]
    c2 = mod_on_coefficients(c2, q)
    c2 = round_on_nearest_integer(c2)
    
    return (c0, c1, c2)    

mult = multiply_ciphertexts(enc_m1, enc_m2, n, p, q)
decrypted = decrypt_poly(mult, ks, n, p, q)

print(f"Expected product: {pr(m1*m2)}")
print(f"Decrypted result: {pr(decrypted)}")

Expected product: 3X**4 - 5X**3 - 4X**2 - 3X - 1
Decrypted result: 3X**4 + 2X**3 + 3X**2 + 4X + 6


#### Relinearisation

As we saw, the multiplication between two ciphertexts resulted in a ciphertext composed by three elements.
This is not a problem di per se, given that we can modify the decryption phase in order to work also on $n$-dimensional ciphertexts. However:

  1. The bigger the ciphertexts, the bigger their memory footprint both in central memory and storage/network;
  2. The bigger the ciphertexts, the bigger the computational load required in performing operations on them.
  
Luckily, BFV allows an operation called *relinearization* which takes a ciphertext of size $k$ and return a ciphertext of size $k-1$ (obviously, at least with dimension $2$).

While the relinearization slightly increases the general value of noise in the ciphertext, it is generally suggested to perform it after every multiplication; also because it requires some keys, which have to be generated by the secret key owner. 

Let $k$ be the dimension of the ciphertexts we want to relinearize (e.g. $3$). Then, to perform the relinearization, we need the relinearization key for $k$, expressed as:
$$\mathbf{evk}_k = ([{-(as+e) +s^{k-1}}]_{\Phi_n, q}, a)$$

Note that if we relinearize after every multiplication, we need just $\mathbf{evk}_3 = ([{-(as+e) +s^2}]_{\Phi_n, q}, a)$.


The actual operation is computed this way: let $c_0, c_1, c_2$ be the elements of the ciphertext we want to relinearize. We also have $\mathbf{evk}_3$. Then, we can compute $(c^{'}_0, c^{'}_1)$ as:

$$c^{'}_0 = c_0 + \mathbf{evk}_3[0]c_2 $$
$$c^{'}_1 = c_1 + \mathbf{evk}_3[1]c_2 $$

The resulting ciphertext is $ct = (c^{'}_0, c^{'}_1)$.

What happens if we try on the result of our last multiplication?  

In [18]:
print(f"Our last ciphertext had {len(mult)} elements.")
print(f"We want to relinearize it to a 2-elements ciphertext.")

# Generate evk3
polynomial_modulus = Poly({0: 1, n: 1})
evk3_0 = pk[0] + (ks * ks)
evk3_0 = divmod(evk3_0, polynomial_modulus)[1]
evk3_0 = mod_on_coefficients(evk3_0, q)

evk3 = (evk3_0, kp[1])

# Relinearize

new_c0 = mult[0] + evk3[0]*mult[2]
new_c1 = mult[1] + evk3[1]*mult[2]

relinearized_mult = (new_c0, new_c1)

Our last ciphertext had 3 elements.
We want to relinearize it to a 2-elements ciphertext.


Are we still able to decrypt it?

In [19]:
decrypted = decrypt_poly(relinearized_mult, ks, n, p, q)

print(f"Expected product: {pr(m1*m2)}")
print(f"Decrypted result: {pr(decrypted)}")

Expected product: 3X**4 - 5X**3 - 4X**2 - 3X - 1
Decrypted result: 2X**15 + 2X**14 + 5X**12 + 5X**11 + X**10 + 5X**9 + 2X**8 + 2X**6 + 4X**5 + X**4 + 6X**3 + 3X + 2


Unfortunately, in the relinearization the term $-e*c_2$ is generated. Given that $c_2$ has coefficients up to $q$, this term will make the decryption fail.

There are advanced solutions to perform the relinearization without having this problem.

### Homomorphic operations between ciphertexts and plaintexts
An important aspect of schemes like BFV is the possibility to compute additions and multiplications not only between ciphertexts, but also between a ciphertext and a plaintext.

This is very important for HE-ML: if a third party wants to run a ML model on encrypted data, it is useless to encrypt the model. Not only, this is actually wasteful because **operations between ciphertexts and plaintexts create much less noise**.

They are quite easy with respect to the ones we saw before.
Let $ct = (ct_0, ct_1, \ldots, ct_n)$ be an encrypted polynomial and $pt$ a plain polynomial.

$$ ct + pt = (ct_0 + \left \lfloor \frac{q}{p} \right \rfloor pt, ct_1, \ldots, ct_n) $$
$$ ct * pt = (ct_0 * pt, ct_1 * pt, \ldots, ct_n * pt) $$

In [20]:
def add_cipher_and_plain(ct, pt, p, q):
    ct = list(ct)
    ct[0] = ct[0] + floor((q/p))*pt
    ct = tuple(ct)
    
    return ct

def multiply_cipher_and_plain(ct, pt):
    
    new_ct = []
    for elem in ct:
        new_ct.append(elem * pt)
    
    return tuple(new_ct)

In [21]:
n = 16
p = 7
q = 424242

ks, kp = generate_keys(n, q)

m1 = Poly({0: 1, 1: 1, 2: 3})
m2 = Poly({0: -1, 1: -2, 2: 1})
print(f"First message: {pr(m1)}")
print(f"Second message: {pr(m2)}")
print(f"Expected sum: {pr(m1+m2)}")
print(f"Expected product: {pr(m1*m2)}")
print(f"-------------------------")

print(f"Encrypting first message...")
enc_m1 = encrypt_poly(m1, kp, n, p, q)

add_result = add_cipher_and_plain(enc_m1, m2, p, q)
mult_result = multiply_cipher_and_plain(enc_m1, m2)

print(f"--------------------------")
print(f"Decrypted sum: {pr(decrypt_poly(add_result, ks, n, p, q))}")
print(f"Decrypted product: {pr(decrypt_poly(mult_result, ks, n, p, q))}")

First message: 3X**2 + X + 1
Second message: X**2 - 2X - 1
Expected sum: 4X**2 - X
Expected product: 3X**4 - 5X**3 - 4X**2 - 3X - 1
-------------------------
Encrypting first message...
--------------------------
Decrypted sum: 4X**2 + 6X
Decrypted product: 3X**4 + 2X**3 + 3X**2 + 4X + 6


In the case of ciphertext-plaintext addition, virtually we are not adding any noise to the ciphertext: in fact, if the plain message is $m_2$, we are adding to the ciphertext $\left \lfloor \frac{q}{p} \right \rfloor m_2$ which will be summed to $\left \lfloor \frac{q}{p} \right \rfloor m_1$. However, adding a polynomial may lead a coefficient in the ciphertext to go above $t$ or also $\frac{q}{2t}$.

For multiplications, the problem is a more complicated; however, generally, the noise incremenet depends on the "dimension" of the plaintext message. This is still an operation which produce much less noise than ciphertext-ciphertext multiplications and do not require a relinearization phase after.

## Encoding
The last - but very important - aspect of BFV (but also of other HE schemes) is the encoding.

In simple words: how can we represent the values we want to encrypt in the most efficient way?

### Scalar encoder
This is the most simple way: if we want to encrypt $42$, encode it as a constant polynomial and proceed if everything we saw till this point.

However, this is very inefficient and we can work only with values < $p$. Increasing $p$ makes the noise increment, so, this encoded is not implemented in SEAL.

### Integer encoder
This is a more efficient way to encode integers in SEAL. Basically, an integer $-(2^n - 1) \leq a \leq (2^n-1)$ can be encoded by computing the binary expansion of $\|a\|$:

$$ \mathbf{IntegerEncode}(a, B=2) = sign(a) \cdot (a_{n-1}x^{n-1} + \ldots + a_1x + a_0) $$

In this case we used the binary expansion, so B=2. It is possible to use other values of B, though.

This has many advantages w.r.t the scalar encoder: the starting coefficients are way smaller, making the noise growth with a lower rate. Also, it allows for a smaller $p$ to be used.

### Fractional encoder
It is also possible to encode fractional values, splitting the integer and fractional part of a value and encoding them in the same polynomials, treating them as integers.

However, BFV was not born with this goal in mind; while it is possible to use this encoder, the CKKS scheme should be used if the use case involves floating values.

### Packing
The packing is different from the last two encoders: its main goal is to encode **multiple** values into a single polynomial. 
This allows the use of SIMD operations on ciphertexts: in fact, multiplying two ciphertexts (operation that already involves SIMD instructions on modern CPUs/GPUs) means multiplying **two vectors** of values.

While this presents many advantages from the computational point of view, it also presents an important limitation: if used, it is no longer possible to access directly the n-th element of an encoded array.

In part this can be solved using an additional operation called *rotation*. This operation rotates the values of an encrypted vector. For example:

$$ [a_0, a_1, \ldots, a_n] $$

becomes:

$$ [a_1, \ldots, a_n, a_0] $$

This requires the generation of *Galois Keys*, one for each rotation step required (e.g. one key for rotations of 1 place, one key for rotations of 2 places, etc). If a Galois key for a particular rotation is not available, a combination of rotation can be performed but this consumes more noise (rotations consume more or less the same noise as relinearization).

In short: use packing when possible, but remembers the limitation of working on array.

# Playground

Remember that the following functions are available:

- ks, kp = generate\_keys(n, q)
- enc_m = encrypt_poly(m, kp, n, p, q)
- decrypted = decrypt_poly(enc_poly, ks, n, p, q)
- enc_sum = add_ciphertexts(enc_m1, enc_m2, n, q)
- enc_mult = multiply_ciphertexts(enc_m1, enc_m2, n, p, q)
- add_result = add_cipher_and_plain(enc_m1, m2, p, q)
- mult_result = multiply_cipher_and_plain(enc_m1, m2)

In [22]:
from numpy.polynomial.polynomial import polyval

In [23]:
n = 16
p = 7
q = 124112

In [24]:
ks, kp = generate_keys(n, q)

In [25]:
print(f"Secret key: {pr(ks)}\n")
print(f"Public key: {pr(kp[0])}, {pr(kp[1])}\n")

Secret key: X**15 - X**14 - X**13 - X**12 + X**11 + X**10 - X**9 - X**8 - X**7 + X**6 - X**5 + X**4 - X**3 - X**2 + 1

Public key: 61199X**15 + 48133X**14 + 104460X**13 + 60601X**12 + 100867X**11 + 115965X**10 + 5255X**9 + 80189X**8 + 20730X**7 + 15550X**6 + 38213X**5 + 39912X**4 + 54197X**3 + 62895X**2 + 113586X + 92746, 80534X**15 + 36864X**14 + 67996X**13 + 93461X**12 + 30336X**11 + 124039X**10 + 109213X**9 + 70976X**8 + 36273X**7 + 39448X**6 + 81173X**5 + 6239X**4 + 70838X**3 + 27318X**2 + 35006X + 72552



In [26]:
m1 = Poly({0: 1, 1: 1, 2: 1})
m2 = Poly({1: 1})

In [27]:
print(f"7: {pr(m1)}")
print(f"2: {pr(m2)}")

7: X**2 + X + 1
2: X


In [28]:
enc_m1 = encrypt_poly(m1, pk, n, p, q)
enc_m2 = encrypt_poly(m2, pk, n, p, q)

In [29]:
print(f"enc_m1: {pr(enc_m1[0])}, {pr(enc_m1[1])}\n")
print(f"enc_m2: {pr(enc_m2[0])}, {pr(enc_m2[1])}\n")

enc_m1: 74404X**15 + 9539X**14 + 102745X**13 + 58368X**12 + 98750X**11 + 74464X**10 + 67822X**9 + 4104X**8 + 23789X**7 + 99591X**6 + 64343X**5 + 61968X**4 + 53390X**3 + 27250X**2 + 64856X + 51090, 121602X**15 + 100582X**14 + 28457X**13 + 103613X**12 + 98904X**11 + 18214X**10 + 3138X**9 + 1039X**8 + 65530X**7 + 11022X**6 + 51615X**5 + 57206X**4 + 51429X**3 + 60289X**2 + 80948X + 13677

enc_m2: 14068X**15 + 6425X**14 + 4887X**13 + 43501X**12 + 64802X**11 + 97740X**10 + 97173X**9 + 106415X**8 + 72200X**7 + 83490X**6 + 37630X**5 + 57126X**4 + 18887X**3 + 23691X**2 + 53327X + 52978, 100091X**15 + 85920X**14 + 55385X**13 + 39411X**12 + 91418X**11 + 96094X**10 + 55480X**9 + 54972X**8 + 87293X**7 + 86329X**6 + 80720X**5 + 68553X**4 + 40828X**3 + 16044X**2 + 51146X + 117168



In [30]:
enc_sum = add_ciphertexts(enc_m1, enc_m2, n, q)
print(f"Sum ciphertext: {pr(enc_sum[0])}, {pr(enc_sum[1])}\n")

decrypted = decrypt_poly(enc_sum, ks, n, p, q)
print(f"Expected sum: {pr(m1+m2)}\n")
print(f"Decrypted result: {pr(decrypted)}\n")

Sum ciphertext: 88472X**15 + 15964X**14 + 107632X**13 + 101869X**12 + 39440X**11 + 48092X**10 + 40883X**9 + 110519X**8 + 95989X**7 + 58969X**6 + 101973X**5 + 119094X**4 + 72277X**3 + 50941X**2 + 118183X + 104068, 97581X**15 + 62390X**14 + 83842X**13 + 18912X**12 + 66210X**11 + 114308X**10 + 58618X**9 + 56011X**8 + 28711X**7 + 97351X**6 + 8223X**5 + 1647X**4 + 92257X**3 + 76333X**2 + 7982X + 6733

Expected sum: X**2 + 2X + 1

Decrypted result: X**2 + 2X + 1



In [31]:
type(decrypted)

numpy.polynomial.polynomial.Polynomial

In [32]:
print(polyval(2, decrypted.coef))

9.0


In [33]:
mult = multiply_ciphertexts(enc_m1, enc_m2, n, p, q)
print(f"Multiplication result: {pr(mult[0])}, {pr(mult[1])}, {pr(mult[2])}\n")
decrypted = decrypt_poly(mult, ks, n, p, q)

print(f"Expected product: {pr(m1*m2)}\n")
print(f"Decrypted result: {pr(decrypted)}\n")

Multiplication result: 4616X**15 + 69851X**14 + 90658X**13 + 27259X**12 + 92777X**11 + 63865X**10 + 46488X**9 + 86233X**8 + 69036X**7 + 13697X**6 + 12395X**5 + 78526X**4 + 87149X**3 + 70424X**2 + 108145X + 97186, 41476X**15 + 15425X**14 + 111259X**13 + 54167X**12 + 4603X**11 + 74182X**10 + 123835X**9 + 67268X**8 + 9265X**7 + 47926X**6 + 6478X**5 + 66494X**4 + 109412X**3 + 13366X**2 + 108332X + 94209, 77259X**15 + 32461X**14 + 9874X**13 + 75310X**12 + 105013X**11 + 101503X**10 + 7418X**9 + 45160X**8 + 24155X**7 + 53139X**6 + 36391X**5 + 76924X**4 + 12497X**3 + 49450X**2 + 5811X + 35650

Expected product: X**3 + X**2 + X

Decrypted result: X**3 + X**2 + X

